In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import KFold,StratifiedKFold

In [2]:
train_df = pd.read_csv('input/train.csv')
# df = pd.read_csv('input/train_min.csv')  # small data
test_df = pd.read_csv('input/test.csv')

In [3]:
X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [4]:
X_test.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [5]:
X.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [6]:
pred_cv = np.zeros(len(X))
pred = np.zeros(len(X_test))

In [7]:
pred_cv

array([0., 0., 0., ..., 0., 0., 0.])

In [8]:
pred

array([0., 0., 0., ..., 0., 0., 0.])

In [25]:
# stacking

models = [
    ('lg', SGDClassifier(loss='log', max_iter=5000, tol=1e-7, alpha=0.3)),
    ('mlp', MLPClassifier(solver='lbfgs', alpha=0.001, hidden_layer_sizes=(5, 2), random_state=1)),
#    ('rf', RandomForestClassifier(n_estimators=10, criterion='entropy')),
#    ('gnb', GaussianNB()),
#    ('qda', QuadraticDiscriminantAnalysis(tol=1e-12)),
]

cv_out = pd.DataFrame(index=train_df.iloc[:,0])
test_out = pd.DataFrame(index=test_df.iloc[:,0])

# add target to cv_out
cv_out['target'] = train_df.iloc[:,1].values

for model in models:
    name = model[0]
    cls = model[1]
    print('working on ' + name)
    
    # k-fold
    n_splits = 5
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    pred_cv = np.zeros(len(X))
    pred = np.zeros(len(X_test))
    
    print(pred_cv)
    
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
        x_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        cls.fit(x_train, y_train)
    
        print(cls.predict_proba(X)[:,1])
        
    
        pred_cv += cls.predict_proba(X)[:,1] / n_splits
        pred += cls.predict_proba(X_test)[:,1] / n_splits
    
    print(pred_cv)
        
    cv_out[name] = pred_cv
    test_out[name] = pred

working on lg
[0. 0. 0. ... 0. 0. 0.]
[0.05828743 0.63476795 0.05225806 ... 0.13435469 0.03725564 0.01298926]
[0.05259577 0.60766548 0.04914548 ... 0.11706492 0.03436046 0.01227349]
[0.05195016 0.62613152 0.05478073 ... 0.12202368 0.0370109  0.01317295]
[0.05769008 0.6288973  0.05197095 ... 0.1253867  0.03438069 0.01306687]
[0.05484528 0.61951717 0.05377239 ... 0.12951671 0.03815319 0.0131894 ]
[0.05507375 0.62339588 0.05238552 ... 0.12566934 0.03623217 0.01293839]
working on mlp
[0. 0. 0. ... 0. 0. 0.]
[0.10049024 0.10049024 0.10049024 ... 0.10049024 0.10049024 0.10049024]
[0.0260693  0.63155002 0.06396419 ... 0.08270687 0.00976465 0.00524465]
[0.10050862 0.10050862 0.10050862 ... 0.10050862 0.10050862 0.10050862]
[0.10049843 0.10049843 0.10049843 ... 0.10049843 0.10049843 0.10049843]
[0.0212923  0.65824827 0.06588232 ... 0.09836624 0.01508993 0.00464254]
[0.06977178 0.31825912 0.08626876 ... 0.09651408 0.06527037 0.0622769 ]


In [26]:
# add statistics
num = len(models);

cv_out['mean'] = cv_out.iloc[:,[1,num]].mean(axis=1)
cv_out['min'] = cv_out.iloc[:,[1,num]].min(axis=1)
cv_out['max'] = cv_out.iloc[:,[1,num]].max(axis=1)

test_out['mean'] = test_out.iloc[:,[0,num-1]].mean(axis=1)
test_out['min'] = test_out.iloc[:,[0,num-1]].min(axis=1)
test_out['max'] = test_out.iloc[:,[0,num-1]].max(axis=1)

In [27]:
cv_out.head()

,target,lg,mlp,mean,min,max
ID_code,,,,,,
train_0,0,0.055074,0.069772,0.062423,0.055074,0.069772
train_1,0,0.623396,0.318259,0.470828,0.318259,0.623396
train_2,0,0.052386,0.086269,0.069327,0.052386,0.086269
train_3,0,0.129259,0.117078,0.123168,0.117078,0.129259
train_4,0,0.144633,0.127758,0.136196,0.127758,0.144633


In [28]:
test_out.head()

,lg,mlp,mean,min,max
ID_code,,,,,
test_0,0.112264,0.101028,0.106646,0.101028,0.112264
test_1,0.224903,0.130917,0.177910,0.130917,0.224903
test_2,0.072041,0.082169,0.077105,0.072041,0.082169
test_3,0.162055,0.149280,0.155668,0.149280,0.162055
test_4,0.075862,0.094445,0.085154,0.075862,0.094445


In [29]:
cv_out.to_csv('input/train_stack.csv')

In [30]:
test_out.to_csv('input/test_stack.csv')